In [7]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
import os

In [8]:
load_dotenv()  # Load environment variables from .env file

True

In [9]:
model = ChatOpenAI()

In [10]:
class BlogState(TypedDict):
    title: str
    outline: str
    content: str

In [11]:
def create_outline(state: BlogState) -> BlogState:
    prompt = PromptTemplate(
        input_variables=["title"],
        template="Create a detailed outline for a blog post titled '{title}'.",
    )
    outline = model.invoke(prompt.format(title=state["title"])).content
    state["outline"] = outline
    return state

In [12]:
def write_content(state: BlogState) -> BlogState:
    prompt = PromptTemplate(
        input_variables=["title", "outline"],
        template="Write a blog post titled '{title}' based on the following outline:\n{outline}",
    )
    content = model.invoke(prompt.format(title=state["title"], outline=state["outline"])).content
    state["content"] = content
    return state

In [22]:
# Define the state graph
graph = StateGraph(BlogState)

# Define the nodes
graph.add_node("create_outline", create_outline)
graph.add_node("write_content", write_content)

# Define the edges
graph.add_edge(START, "create_outline")
graph.add_edge("create_outline", "write_content")
graph.add_edge("write_content", END)

# Create the workflow
workflow = graph.compile()

In [16]:
# Initialize and execute the workflow
initial_state = {"title": "The Future of Diesel Cars"}
final_state = workflow.invoke(initial_state)
print(final_state["content"])

The Future of Diesel Cars

In the realm of the automotive industry, diesel cars have played a significant role in providing consumers with a fuel-efficient and powerful option for their daily transportation needs. With a rich history dating back to the early 20th century, diesel cars have evolved to become a staple in the market, offering a unique driving experience and economic benefits to drivers. However, in recent years, the perception of diesel cars has shifted due to growing concerns about their impact on the environment and public health. In this blog post, we will explore the future of diesel cars and the potential trends and developments that lie ahead.

Environmental concerns have placed a spotlight on diesel cars, particularly in relation to emissions and air pollution. Diesel engines have been notorious for their high levels of nitrogen oxides and particulate matter emissions, which contribute to poor air quality and have detrimental effects on human health. In response to 

In [27]:
from IPython.display import Image
Image(workflow.get_graph().draw_mermaid_png())

ValueError: Failed to reach https://mermaid.ink/ API while trying to render your graph. Status code: 502.

To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`

In [24]:
workflow.get_graph().draw_mermaid()

'---\nconfig:\n  flowchart:\n    curve: linear\n---\ngraph TD;\n\t__start__([<p>__start__</p>]):::first\n\tcreate_outline(create_outline)\n\twrite_content(write_content)\n\t__end__([<p>__end__</p>]):::last\n\t__start__ --> create_outline;\n\tcreate_outline --> write_content;\n\twrite_content --> __end__;\n\tclassDef default fill:#f2f0ff,line-height:1.2\n\tclassDef first fill-opacity:0\n\tclassDef last fill:#bfb6fc\n'